In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [6]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/rnn_conv/256rnn/' #3denses/bio_neurons/'
#'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [5] #[1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
#architecture = "only_denses"
#architecture = "conv_dense"
architecture = "rnn_conv"
#architecture = "stack_conv"

In [7]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [8]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [10]:
#model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
model = building_convrnn_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
#model = building_stackconv_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 10)          1310      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 100, 10)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 10)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               546816    
_________________________________________________________________
dense_3 (Dense)              (None, 313)               160569    
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1000, 26)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1000, 10)          1310      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 100, 10)           0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 10)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               546816    
_________________________________________________________________
dense_6 (Dense)              (None, 313)               160569    
__________

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



138/138 [==============================] - 42s 306ms/step - loss: 0.7472 - acc: 0.1757 - val_loss: 0.2614 - val_acc: 0.1003

Epoch 00079: val_acc did not improve from 0.11047
Epoch 80/200
138/138 [==============================] - 42s 305ms/step - loss: 0.7466 - acc: 0.1766 - val_loss: 0.2640 - val_acc: 0.1003

Epoch 00080: val_acc did not improve from 0.11047
Epoch 81/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7490 - acc: 0.1751 - val_loss: 0.2616 - val_acc: 0.1003

Epoch 00081: val_acc did not improve from 0.11047
Epoch 82/200
138/138 [==============================] - 42s 306ms/step - loss: 0.7453 - acc: 0.1767 - val_loss: 0.2619 - val_acc: 0.1003

Epoch 00082: val_acc did not improve from 0.11047
Epoch 83/200
138/138 [==============================] - 42s 305ms/step - loss: 0.7443 - acc: 0.1767 - val_loss: 0.2598 - val_acc: 0.1003

Epoch 00083: val_acc did not improve from 0.11047
Epoch 84/200
138/138 [==============================] - 42s 306ms/step - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



138/138 [==============================] - 42s 306ms/step - loss: 0.7136 - acc: 0.1809 - val_loss: 0.2452 - val_acc: 0.1169

Epoch 00153: val_acc did not improve from 0.12771
Epoch 154/200
138/138 [==============================] - 42s 306ms/step - loss: 0.7157 - acc: 0.1798 - val_loss: 0.2473 - val_acc: 0.1169

Epoch 00154: val_acc did not improve from 0.12771
Epoch 155/200
138/138 [==============================] - 42s 306ms/step - loss: 0.7160 - acc: 0.1817 - val_loss: 0.2461 - val_acc: 0.1149

Epoch 00155: val_acc did not improve from 0.12771
Epoch 156/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7164 - acc: 0.1817 - val_loss: 0.2477 - val_acc: 0.1143

Epoch 00156: val_acc did not improve from 0.12771
Epoch 157/200
138/138 [==============================] - 42s 305ms/step - loss: 0.7133 - acc: 0.1804 - val_loss: 0.2459 - val_acc: 0.1175

Epoch 00157: val_acc did not improve from 0.12771
Epoch 158/200
138/138 [==============================] - 42s 306ms/st

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



138/138 [==============================] - 42s 305ms/step - loss: 0.7113 - acc: 0.1832 - val_loss: 0.2457 - val_acc: 0.1245

Epoch 00134: val_acc did not improve from 0.12835
Epoch 135/200
138/138 [==============================] - 42s 305ms/step - loss: 0.7119 - acc: 0.1812 - val_loss: 0.2453 - val_acc: 0.1220

Epoch 00135: val_acc did not improve from 0.12835
Epoch 136/200
138/138 [==============================] - 43s 308ms/step - loss: 0.7106 - acc: 0.1826 - val_loss: 0.2437 - val_acc: 0.1232

Epoch 00136: val_acc did not improve from 0.12835
Epoch 137/200
138/138 [==============================] - 42s 306ms/step - loss: 0.7104 - acc: 0.1802 - val_loss: 0.2466 - val_acc: 0.1226

Epoch 00137: val_acc did not improve from 0.12835
Epoch 138/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7121 - acc: 0.1820 - val_loss: 0.2469 - val_acc: 0.1194

Epoch 00138: val_acc did not improve from 0.12835
Epoch 139/200
138/138 [==============================] - 42s 307ms/st

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



138/138 [==============================] - 42s 306ms/step - loss: 0.7565 - acc: 0.1846 - val_loss: 0.2881 - val_acc: 0.0964

Epoch 00189: val_acc did not improve from 0.12835
Epoch 190/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7496 - acc: 0.1841 - val_loss: 0.2859 - val_acc: 0.0964

Epoch 00190: val_acc did not improve from 0.12835
Epoch 191/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7520 - acc: 0.1837 - val_loss: 0.2823 - val_acc: 0.0964

Epoch 00191: val_acc did not improve from 0.12835
Epoch 192/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7484 - acc: 0.1832 - val_loss: 0.2807 - val_acc: 0.0964

Epoch 00192: val_acc did not improve from 0.12835
Epoch 193/200
138/138 [==============================] - 42s 307ms/step - loss: 0.7488 - acc: 0.1855 - val_loss: 0.2786 - val_acc: 0.0964

Epoch 00193: val_acc did not improve from 0.12835
Epoch 194/200
138/138 [==============================] - 42s 307ms/st

138/138 [==============================] - 44s 317ms/step - loss: 0.7568 - acc: 0.1759 - val_loss: 0.2810 - val_acc: 0.1003

Epoch 00037: val_acc did not improve from 0.10026
Epoch 38/200
138/138 [==============================] - 44s 317ms/step - loss: 0.7511 - acc: 0.1766 - val_loss: 0.2770 - val_acc: 0.1003

Epoch 00038: val_acc did not improve from 0.10026
Epoch 39/200
138/138 [==============================] - 44s 316ms/step - loss: 0.7534 - acc: 0.1763 - val_loss: 0.2780 - val_acc: 0.1003

Epoch 00039: val_acc did not improve from 0.10026
Epoch 40/200
138/138 [==============================] - 44s 316ms/step - loss: 0.7518 - acc: 0.1770 - val_loss: 0.2755 - val_acc: 0.1003

Epoch 00040: val_acc did not improve from 0.10026
Epoch 41/200
138/138 [==============================] - 44s 318ms/step - loss: 0.7545 - acc: 0.1773 - val_loss: 0.2771 - val_acc: 0.1003

Epoch 00041: val_acc did not improve from 0.10026
Epoch 42/200
138/138 [==============================] - 44s 318ms/step - 